<a href="https://colab.research.google.com/github/amine406/SeoProject/blob/main/Crawling_Multi_thread%C3%A9_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Imports
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Initialiser les variables
links = set()  # Suivre les URLs traitées
page_info = []  # Stocker les données SEO pour chaque URL

# Créer une session globale
session = requests.Session()

# Ajouter un User-Agent
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
})

# Fonction pour vérifier si l'URL est valide et interne
def is_valid_url(_url, base_url):
    parsed_url = urlparse(_url)
    return bool(parsed_url.scheme) and bool(parsed_url.netloc) and base_url in _url

# Fonction pour crawler et analyser la page
def crawl_page(_url, base_url):
    try:
        response = session.get(_url, timeout=5)  # Timeout après 5 secondes
        http_status = response.status_code
        print(f"Crawling {_url} {http_status}")

        if http_status != 200:
            return []  # Ignorer si le statut n'est pas OK

# Vérifier que la réponse est bien du HTML
        content_type = response.headers.get("Content-Type", "")
        print(f"Content-Type de {_url}: {content_type}")
        if "text/html" not in content_type:
            return []  # Ignorer le contenu non-HTML

# Utiliser un parser
        soup = BeautifulSoup(response.content, "html.parser")

# Extraire les informations SEO
        title = soup.title.string if soup.title else "No title"
        description = soup.find("meta", attrs={"name": "description"})
        description = description["content"] if description else "No description"
        h1 = soup.find("h1")
        h1_text = h1.get_text() if h1 else "No H1"

# Compter le nombre de liens sur la page
        all_links = soup.find_all("a", href=True)
        link_count = len(all_links)
        print(f"{link_count} liens trouvés sur {_url}")

# Trouver les nouveaux liens internes
        inew_links = [urljoin(base_url, link.get('href')) for link in all_links if is_valid_url(urljoin(base_url, link.get('href')), base_url)]

 # Ajouter les informations SEO et les nouveaux liens à la liste
        page_info.append({
            "URL": _url,
            "HTTP Status": http_status,
            "Title": title,
            "Description": description,
            "H1": h1_text,
            "Number of Links": link_count,
            "New Links Found": "\n".join(inew_links)  # Retour à la ligne pour chaque nouveau lien
        })

        return inew_links

    except requests.RequestException as e:
        print(f"Échec du crawl de {_url}: {e}")
        return []  # Retourner une liste vide en cas d'erreur

# Fonction principale pour démarrer le crawl du site
def crawl_website(start_url, max_threads=5):
    base_url = "{0.scheme}://{0.netloc}".format(urlparse(start_url))
    pending_urls = [start_url]

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        crawl = []

        while pending_urls:
            _url = pending_urls.pop()

            if _url not in links:
                links.add(_url)
                crawl.append(executor.submit(crawl_page, _url, base_url))

        for job in crawl:
            inew_links = job.result()
            print(f"Nouveaux liens trouvés : {inew_links}")
            pending_urls.extend(inew_links)

# Convertir les données  en DataFrame
    data_frame = pd.DataFrame(page_info)

    print("\nTableau récapitulatif des pages crawlées :")
    print(data_frame)

# Exporter vers un fichier CSV si des données ont été collectées
    if not data_frame.empty:
        data_frame.to_csv("seo_crawl_results.csv", index=False)
        print("Résultats exportés vers 'seo_crawl_results.csv'.")
    else:
        print("Aucune donnée à exporter.")

# Démarrer le crawler sur le site
if __name__ == "__main__":
    print("Démarrage du crawl sur https://aundetailpres.fr...")
    crawl_website("https://aundetailpres.fr", max_threads=20)
    print("Crawl terminé.")


Démarrage du crawl sur https://aundetailpres.fr...
Crawling https://aundetailpres.fr 200
Content-Type de https://aundetailpres.fr: text/html; charset=UTF-8
289 liens trouvés sur https://aundetailpres.fr
Nouveaux liens trouvés : ['https://aundetailpres.fr#content', 'https://aundetailpres.fr/', 'https://aundetailpres.fr/category/mariage-2/', 'https://aundetailpres.fr/category/mariage-2/decoration-et-sortie-deglise/', 'https://aundetailpres.fr/category/mariage-2/astuces-organisation-et-animation/', 'https://aundetailpres.fr/category/mariage-2/toute-la-decoration/', 'https://aundetailpres.fr/category/mariage-2/faire-part-mariage-2/', 'https://aundetailpres.fr/category/mariage-2/marque-place-menus-et-plan-de-table/', 'https://aundetailpres.fr/category/mariage-2/livre-dor-et-urne/', 'https://aundetailpres.fr/category/mariage-2/livret-de-messe-mariage-2/', 'https://aundetailpres.fr/category/mariage-2/textes-pour-la-ceremonie/', 'https://aundetailpres.fr/category/mariage-2/tout-pour-la-mariee/